In [2]:
import math
import time
import pandas as pd
import numpy as np

In [3]:
def Ccoset5M8(x,y,z,s=False):
    if not s:
        f = math.gcd(x,math.gcd(y,z))
        if z<0:
            f = -f
        x = x//f
        y = y//f
        z = z//f
    if z == 1:
        x, y = 2*x, 2*y
    if x%4 == y%4:
        if y%2 == 0:
            return "Zero coset"
        else:
            return 1
    else:
        if y%2 == 0:
            return 0
        else:
            return 2 
def CFFcoset5M8(x,y,z):
    if z<0:
        x,y,z = -x,-y,-z
    if z%4==0:
        return "Outside domain"
    if z%2==0:
        return Ccoset5M8(x,y,2,True)
    else:
        return Ccoset5M8(x,y,1,True)

def idealProduct(D,Q1,P1,Q2,P2):
    r = 2
    Q1onr = Q1//r
    Q2onr = Q2//r
    P12onr = (P1+P2)//r
    V,W,Y,S = abcsolve(Q1onr,Q2onr,P12onr)
    Q1onS = Q1//S
    Q2onrS = Q2onr//S
    Q = Q1onS*Q2onrS
    R2 = (D-P2**2)//Q2
    U = (W*(P1-P2)+Y*R2)%(Q1onS)
    P = (P2+U*Q2onrS)%Q
    return S, Q, P    

def extgcd(a,b):
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        q = old_r // r
        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t
    return old_s,old_t,old_r

def abcsolve(a,b,c):
    n,m,g1 = extgcd(a,b)
    x,y,g2 = extgcd(g1,c)
    return(x*n,x*m,y,g2)

def squarefree(n):
    for i in range(2,math.ceil(math.sqrt(n))+1):
        if n%(i**2)==0:
            return False
    return True

def prime(n):
    if n==1:
        return False
    for i in range(2,math.floor(math.sqrt(n))+1):
        if n%i==0:
            return False
    return True

In [69]:
def NUCOMPtest(D,Q1,P1,Q2,P2):
    r = 2 #sigma
    if Q1 <= Q2:
        Q1, Q2 = Q2, Q1
        P1, P2 = P2, P1
    Q1onr = Q1//r
    Q2onr = Q2//r
    s, X, G = extgcd(Q1onr,Q2onr)
    X = X%(Q1onr)
    Y, Z, S = extgcd((P1+P2)//r,G)
    R2 = (D-P2**2)//Q2
    U = (X * Z * (P1-P2) + Y * R2)%(Q1//S)
    bm2 = 0
    bm1 = Q1//S #R_-1
    b0 = U #R_0
    am1 = 0 #C_-1
    a0 = -1 #C_0
    cm1 = (Q2//r)//S
    c0 = (P1-P2)//bm1
    dm2 = 0
    dm1 = P1+P2
    d0 = ((P1+P2)*b0-r*S*R2)//bm1
    Am2 = 0
    Am1 = 1
    Bm2 = 1
    Bm1 = 0
    q = ""
    lastq = ""
    i = -1
    Qprime = ""
    Pprime = ""
    print("i","\t","b_i","\t","q_i","\t","A_i","\t","B_i","\t","q_i'","\t","A_i'","\t","B_i'","\t","a_i","\t","c_i","\t","d_i","\t","Q_i'","P_i'")
    print("-2\t\t\t0\t1\t\t\t\t\t\t\t\t")
    print(i,"\t",bm1,"\t",q,"\t",Am1,"\t",Bm1,"\t",lastq,"\t",Bm2,"\t",Am2,"\t",am1,"\t",cm1,"\t",dm1,"\t",Qprime,"\t",Pprime)
    while b0 != 0:
        i = i + 1
        lastq, q = q, math.floor(bm1/b0) #q_j
        Qprime = (-1)**i * (bm1*cm1 - am1*dm1)
        Pprime = (-1)**i * (bm2*cm1 - am1*dm2) + P2
        bm2, bm1, b0 = bm1, b0, bm1 - q * b0 #b_j, b_{j-1}
        Am2, Am1 = Am1, q * Am1 + Am2
        Bm2, Bm1 = Bm1, q * Bm1 + Bm2
        am1, a0 = a0, am1 - q * a0
        cm1, c0 = c0, cm1 - q * c0
        dm2, dm1, d0 = dm1, d0, dm1 - q * d0
        
        print(i,"\t",bm1,"\t",q,"\t",Am1,"\t",Bm1,"\t",lastq,"\t",Bm2,"\t",Am2,"\t",am1,"\t",cm1,"\t",dm1,"\t",Qprime,"\t",Pprime)

In [4]:
def NUCOMP(D,Q1,P1,Q2,P2):
    floorRootD = math.floor(D**(1/2))
    r=2
    if Q1 <= Q2:
        Q1, Q2 = Q2, Q1
        P1, P2 = P2, P1
    Q1onr = Q1//r
    Q2onr = Q2//r
    s, X, G = extgcd(Q1onr,Q2onr)
    X = X%(Q1onr)
    Y, Z, S = extgcd((P1+P2)//r,G)
    R2 = (D-P2**2)//Q2
    U = (X * Z * (P1-P2) + Y * R2)%(Q1//S)
    Rm1 = U
    R0 = Q1//S
    Cm1 = -1
    C0 = 0
    i = -1
    f = 2*(D**(1/4))
    if R0 < f:
        newQ = Q1*Q2//(r*S**2) #Q_i+1
        newP = (P2+U*Q2//(r*S))%newQ #P_i+1
    else:
        while R0 > f:
            i += 1
            q = math.floor(Rm1/R0) #q_i = R_i-2/R_i-1
            Cm1, C0 = C0, Cm1 - q*C0 #Cm1 is now C_i-1, C0 is now C_i
            Rm1, R0 = R0, Rm1 - q*R0 #Rm1 is now R_i-1, R0 is now R_i
        M1 = ((Q2onr//S)*R0+(P1-P2)*C0)//(Q1//S)
        M2 = ((P1+P2)*R0+r*S*R2*C0)//(Q1//S)
        newQ = ((-1)**(i+1))*(R0*M1-C0*M2)
        newP = ((Q2onr//S)*R0+newQ*Cm1)//C0 - P2
    newQprime = abs(newQ)
    k = math.floor((floorRootD-newP)/newQprime)
    newPprime = k*newQprime+newP
    Bm1 = abs(Cm1)
    B0 = abs(C0)
    reps=0
    while True:
        if newPprime + floorRootD >= newQprime:
            return newQprime, newPprime%newQprime, S*(newQ*Bm1+newP*B0), -1*S*B0, newQ
        #print(reps,newQ,newP)
        reps = reps+1
        q = math.floor((newP+floorRootD)/newQprime)
        newP = q * newQprime - newP
        newQ = (D-newP**2)//newQprime
        newQprime = abs(newQ)
        k = math.floor((floorRootD-newP)/newQprime)
        newPprime = k*newQprime+newP
        Bm1, B0 = B0, (q * B0 + Bm1)%4

In [5]:
#put type = "r" for finding a reduced ideal
#put type = "b" for finding the baby steps
#put type = "f" for the full expansion
def reduceQP(D,Q,P,type):
    rootD = math.sqrt(D)
    floorRootD = math.floor(rootD)
    floorRootD32 = math.floor(rootD*3/2)
    sD = 2
    q0 = (P+floorRootD)//Q
    i = 1
    flag = False
    c = 0
    if type == "r":
        Ctheta = 0
        while (i == 1 or c<1):
            #if Q < floorRootD and 0 < Q:
            if Q < floorRootD32 and 0 < Q:
                flag = True
            if flag:
                c += 1
            P = q0*Q-P #P_i+1
            Cpsi = CFFcoset5M8(P,1,Q)
            Qtest = D/Q - (P/Q)*P
            Q = (D-P**2)//Q #Q_i+1
            #print(Q,P)
            if round(Qtest)!=Q:
                print("Qs not equal!")
            Ctheta = (Ctheta+Cpsi)%3
            q0 = (P+floorRootD)//Q #q_i+1
            i = i + 1
        return Q,P,Ctheta
    elif type == "nr":
        Ctheta = 0
        while True:
            P = q0*Q-P #P_i+1
            Cpsi = CFFcoset5M8(P,1,Q)
            Q = (D-P**2)//Q #Q_i+1
            #print(Q,P)
            Ctheta = (Ctheta+Cpsi)%3
            q0 = (P+floorRootD)//Q #q_i+1
            aq = abs(Q)
            pmaq = P%aq
            if pmaq + floorRootD >= aq:
                return aq, pmaq, Ctheta
    elif type == "b":
        theta = 0
        Q0 = Q
        RBm2, RBm1, RB0, RB1 = 0,0,0,1
        RG0, RG1 = Q%4,(Q*q0-P)%4
        CQPdict = {}
        endEarlyFlag = False
        fourthRootD = math.pow(D,1/4)
        CQPdict[str(Q)+" "+str(P%Q)] = 0
        #print(q0,Q,P)
        while (i == 1 or c < 3):
            lastP, P = P, q0*Q-P #P_i,i+1
            lastQ, Q = Q, (D-P**2)//Q #Q_i,i+1
            q0 = (P+floorRootD)//Q #q_i+1
            #print(q0,Q,P)
            RG0, RG1 = RG1, (P*RB1+Q*RB0)%4 #G_i-1,i
            RBm2, RBm1, RB0, RB1 = RBm1, RB0, RB1, (q0*RB1+RB0)%4 #B_i-2,i-1,i,i+1
            if P==lastP:
                endEarlyFlag = True
                correctIdeal = []
                cReducedTheta = Ccoset5M8(RG1*RBm1+RG0*RBm2,RBm1*(RB0+RBm2),Q0)
                break
            if Q==lastQ:
                endEarlyFlag = True
                correctIdeal = []
                cReducedTheta = Ccoset5M8(RG1*RB0+RG0*RBm1,RB0**2+RBm1**2,Q0)
                break
            cyclicTheta = Ccoset5M8(RG1,RB0,Q0)
            i = i + 1
            CQPdict[str(Q)+" "+str(P%Q)] = cyclicTheta
            theta += math.log((P+rootD)/lastQ)
            if theta > fourthRootD:
                flag = True
            if flag:
                c += 1
            if c == 1:
                correctIdeal = [Q,P%Q]
                cReducedTheta = cyclicTheta
        return CQPdict, correctIdeal, cReducedTheta, endEarlyFlag,i
    elif type == "f":
        Q0 = Q
        Bm1, B0, B1 = 0,0,1
        while (i == 1 or Q!=sD):
            P = q0*Q-P #P_i+1
            Q = (D-P**2)//Q #Q_i+1
            #print(Q,P)
            q0 = (P+floorRootD)//Q #q_i+1
            Bm1, B0, B1 = B0, B1, (q0*B1+B0)%4 #B_i, B_i+1
            i = i + 1
        #print(D,i)
        G1 = (P*B0+Q*Bm1)%4
        theta = Ccoset5M8(G1,B0,Q0)
        return theta

In [9]:
def find_FUnit(D):
    P = 1
    Q = 2
    Cdict, firstQP, Ctheta, eef,p = reduceQP(D,Q,P,"b")
    print(Cdict,firstQP)
    if eef:
        return Ctheta
    firstCTheta = Ctheta
    firstQ, firstP = firstQP[0], firstQP[1]
    newQ, newP = firstQ, firstP
    i = 0
    l = set(Cdict.keys())
    #print("\nUsing infrastructure:")
    while ((i==0 or str(newQ)+" "+str(newP%newQ) not in l)):
        #print(i)
        print(D,newQ,newP,firstQ,firstP)
        newQ,newP,testA,testB,testC = NUCOMP(D,newQ,newP,firstQ,firstP)
        print(newQ,newP,testA,testB,testC)
        #S, newQ, newP = idealProduct(D,newQ,newP,firstQ,firstP)        
        #Ctheta = (firstCTheta+Ctheta)%3
        #newQ,newP,Cd = reduceQP(D,newQ,newP,"r")
        #print("GAUSS:",newQ,newP%newQ,Cd)
        #print("NUCOMP:",testQ,testP,testA,testB,testC)
        #print(testA,testB,testC)
        Cd = CFFcoset5M8(testA,testB,testC)
        Ctheta = (Ctheta-Cd+firstCTheta)%3
        i+=1
        if i>1000:
            return -999
    Cfunit = (Ctheta - Cdict[str(newQ)+" "+str(newP%newQ)])%3
    return Cfunit

In [7]:
r = range(13,10000,8)
r = [i for i in r if squarefree(i)]

In [10]:
find_FUnit(1461)

{'2 1': 0, '46 37': 1, '30 9': 2, '34 21': 0, '38 13': 1, '22 3': 2, '50 19': 1, '10 1': 0, '62 29': 1} [50, 19]
1461 50 19 50 19
62 33 281 -1 62
1461 62 33 50 19
30 21 -219 -1 -30
1461 30 21 50 19
30 9 150 0 30


1

In [45]:
answers = []
t1 = time.time()
for i in r:
    try:
        Cf = find_FUnit(i)
        Ff = reduceQP(i,2,1,"f")
        answers.append([i,Cf])
    except:
        print(i,"Failed")
t2 = time.time()
print(t2-t1)

0.0781395435333252


In [49]:
pd.DataFrame(answers).to_csv("correct.csv")

In [157]:
t1 = time.time()
for i in r:
    try:
        Rf = reduceQP(i,2,1,"f")
        print(i,Rf)
    except:
        print(i,"Failed")
t2 = time.time()
print(t2-t1)

13 2
21 1
29 1
37 0
53 2
61 2
69 2
77 1
85 1
93 2
101 0
109 1
133 2
141 0
149 1
157 1
165 1
173 1
181 1
197 0
205 1
213 1
221 2
229 2
237 1
253 1
269 0
277 1
285 1
293 1
301 2
309 2
317 1
341 2
349 0
357 2
365 2
373 0
381 0
389 0
397 2
413 2
421 2
429 2
437 1
445 1
453 2
461 1
469 2
485 0
493 2
501 1
509 1
517 1
533 2
541 1
557 0
565 1
573 0
581 2
589 1
597 2
613 2
629 1
645 2
653 1
661 2
669 2
677 0
685 2
701 0
709 0
717 1
733 2
741 1
749 1
757 0
773 2
781 0
789 1
797 2
805 1
813 0
821 1
829 0
853 2
861 1
869 2
877 0
885 0
893 1
901 0
917 2
933 0
941 2
949 1
957 2
965 2
973 0
989 2
997 0
0.002000093460083008


In [7]:
df = pd.DataFrame(answers)
df.to_csv("C:\\Users\\james\\Documents\\2024\\Honours\\RFU-P.csv")